<a href="https://colab.research.google.com/github/MatteoZancanaro-5758278/M_Zancanaro-Programming-BigDataAnalytics/blob/main/Assignment4/4_01_Linear_Regression.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 4.01 Linear Regression (statistics vs. machine learning)

In [7]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns

#package/library for the traditional implementation
import statsmodels.api as sm

#package/library for ML
from sklearn.linear_model import LinearRegression, Lasso

#error metrics to asses the model
from sklearn.metrics import mean_squared_error, r2_score

df = pd.read_csv("https://raw.githubusercontent.com/jbrownlee/Datasets/master/housing.csv", header=None)
df.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13
0,0.00632,18.0,2.31,0,0.538,6.575,65.2,4.0900,1,296.0,15.3,396.90,4.98,24.0
1,0.02731,0.0,7.07,0,0.469,6.421,78.9,4.9671,2,242.0,17.8,396.90,9.14,21.6
2,0.02729,0.0,7.07,0,0.469,7.185,61.1,4.9671,2,242.0,17.8,392.83,4.03,34.7
3,0.03237,0.0,2.18,0,0.458,6.998,45.8,6.0622,3,222.0,18.7,394.63,2.94,33.4
4,0.06905,0.0,2.18,0,0.458,7.147,54.2,6.0622,3,222.0,18.7,396.90,5.33,36.2


In our data set we have 13x $x$ values (independent variables/features) as columns 0-12, and one $Y$ value (dependent variable/target) as column 13. We'll seperate that out ready for modelling:

In [8]:
x_values = df.drop([13], axis=1)
print(x_values.head())
print("\n")

y_values = df[13]
print(y_values.head())

        0     1     2   3      4      5     6       7   8      9     10  \
0  0.00632  18.0  2.31   0  0.538  6.575  65.2  4.0900   1  296.0  15.3   
1  0.02731   0.0  7.07   0  0.469  6.421  78.9  4.9671   2  242.0  17.8   
2  0.02729   0.0  7.07   0  0.469  7.185  61.1  4.9671   2  242.0  17.8   
3  0.03237   0.0  2.18   0  0.458  6.998  45.8  6.0622   3  222.0  18.7   
4  0.06905   0.0  2.18   0  0.458  7.147  54.2  6.0622   3  222.0  18.7   

       11    12  
0  396.90  4.98  
1  396.90  9.14  
2  392.83  4.03  
3  394.63  2.94  
4  396.90  5.33  


0    24.0
1    21.6
2    34.7
3    33.4
4    36.2
Name: 13, dtype: float64


In [9]:
mod = sm.OLS(y_values, x_values)
res= mod.fit()
print(res.summary())

                                 OLS Regression Results                                
Dep. Variable:                     13   R-squared (uncentered):                   0.959
Model:                            OLS   Adj. R-squared (uncentered):              0.958
Method:                 Least Squares   F-statistic:                              891.3
Date:                Mon, 10 Nov 2025   Prob (F-statistic):                        0.00
Time:                        09:36:39   Log-Likelihood:                         -1523.8
No. Observations:                 506   AIC:                                      3074.
Df Residuals:                     493   BIC:                                      3128.
Df Model:                          13                                                  
Covariance Type:            nonrobust                                                  
                 coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------

#### Observations
Omnibus and JB check for the normality in the residual distribution, Omnibus reject the null hp of normality and JB too

--> residual are not symmetric(probably outlers)(unbiased coeff and SE)

--> non linearity(poor predictions)

Durbin Watson test for autocorrelation in residuals has a strong positive autocorrelation
- 2--> no corr
- < 2--> positive(patterns)
- > 2 --> negative

--> omitted variable?

--> autocorr?

Cond. No. extremely high --> multicollinearity among predictors
- < 30 --> good
- 30-100 moderate mult
- > 1000 severe mult

---> redundant or correlated predictors(coeff unstable, small changes affect est.coeff. SE inflate)


In [10]:
#split training test
from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(x_values, y_values, test_size=0.2, random_state=1234)

print(X_train.shape)
print(X_test.shape)
print(Y_train.shape)
print(Y_test.shape)

(404, 13)
(102, 13)
(404,)
(102,)


###Regularisation
Now we can turn our attention to the model. Our linear regression will remain the same, but we will use a modified algorithm to determine it. Specifically we will use $L1$ regression (LASSO) where the objective function (cost/loss function) is:

$OLS\ objective + \alpha \cdot  \Sigma |\beta_i|$

We add a penalty to the ordinary least squares (OLS) objective (described above) to limit the sum of the absolute values of the co-efficients ($\beta$ weights). Absolute values, denoted by the "|" symbols in $|\beta|$, means that we change any negatives values to their positive equivalent - e.g. -2 becomes 2; -1 becomes 1; while 4 stays as 4.

Because OLS is a _minimisation_ objective (we want the line with the smallest error), this means we will be minimising the sum ($\Sigma$) of the $\beta$ weights as well as OLS. I.e. we want a line that can closely fit the data whilst not having any particularly large and noisy weights. If we have a large weight for $\beta_n$, this means that any random errors in $x_n$ will be magnified, as we are multiplying two, meaning our model can have more variance.

We can see the amount of influence this penalty has is controlled by the hyperparameter $\alpha$. If we had a very large value of $\alpha$, the model will be encouraged to reduce all $\beta$ weights towards zero; if we have a very small value of $\alpha$ (approaching zero), the model will be basically the same as normal linear regression (i.e. the penalty term is ignored). Hyperparameters are something we will return to later in the module but for the moment we will just use an arbitrary value of 0.25.

With this in mind we can learn our model from the data:

In [11]:
# configure the model
l1_algo = Lasso(alpha=0.25)
l1_algo

Lasso(alpha=0.25)

In [15]:
# learn the mdoel from training data
l1_model = l1_algo.fit(X_train, Y_train)

#predict the data
predict = l1_model.predict(X_test)

#calculate R^2 by comparing the predicted values and real values
r2 = r2_score(Y_test,predict)
mse = mean_squared_error(Y_test,predict)
print(f"R^2 score si {round(r2,2)} and MSE {round(mse,3)}")

R^2 score si 0.76 and MSE 24.997


We can see that $R^2$ is lower than our previous (statistical) model which achieved 96%. However, we need to recall that in this case we are testing the model against previously unseen data ... a much harder problem. Overall an $R^2$ at 76% is generally going to be considered quite high for this kind of task - so good news. That is not to say that this is the "optimal" model, and in practice we would experiment further (for example, trying different values of $\alpha$ to control how much our model favours OLS and the line of least error, versus how much we penalise large $\beta$ weights with the L1 penalty).

You may also notice the conspicuous absence of any hypothesis testing ($p$-values). Typically we would not report or test these in a machine learning approach. We do not have the same underlying (implicit) beliefs of the model as generator of the data, and ultimately no hypotheses to test (in the traditional sense). In essence the model is the hypothesis (our belief that this model will be able to score ($R^2$ in this case) reasonably well on unseen/test data). In machine learning terminology, you will often hear the possible models to be tested described as the 'hypothesis space'. Given this change in focus, $p$-values are simply irrelevant.

However, we may also want to inspect the $\beta$ values of the model:

In [17]:
# print the beta values of the model(co-coefficients)
betas_l1 = l1_model.coef_
counter=0
for col in x_values.columns:
  if counter==0:
    print("Beta weights/co-efficients - LASSO")
    print("-----------------------------------------")
  print(f"{col} : {round(betas_l1[counter], 4)}")
  counter +=1

Beta weights/co-efficients - LASSO
-----------------------------------------
0 : -0.0831
1 : 0.0657
2 : -0.0228
3 : 0.0
4 : -0.0
5 : 2.3925
6 : -0.0087
7 : -1.2545
8 : 0.305
9 : -0.0165
10 : -0.7983
11 : 0.011
12 : -0.6034


We may observe two things. Firstly we can see the $\beta$ weights for feature 3 and 4 have been set at zero - effectively deleting these features entirely (any number multipled by zero is zero). From the class metaphor, the algorithm has decided these two decision makers are not improving the decision quality and their influence increases the variance of the model.

Secondly, we will notice that nearly all of the $\beta$ weights are smaller (closer to zero) than in the statistical appraoch of the last Notebook. For instance, feature 5 had a $\beta$ weight of 5.93 in the original model, compared with 2.39 here (less than half). Again, this fits with our metaphor - the algorithm has tried to ensure that no voices are too loud in  our group discussion. I.e. our model will be less likely to over-respond to excess noise in any particular feature as the associated $\beta$ weight is smaller.

So there we have it. Hopefully the comparison of the two approaches gives you some insight into how these two methodologies differ, and how the statistical approach has effectively been adapted to the supervised machine learning problem ... building models that can learn from data.

In [23]:
# for better lasso uses we should have an optimized lasso through cross-validation

from sklearn.linear_model import LassoCV

#automatically choose best alpha using cross-validation
lasso_cv = LassoCV(cv=5, random_state=0)
lasso_cv.fit(X_train,Y_train)

print("Optimal alpha:", lasso_cv.alpha_)
print("\nCoefficients:", lasso_cv.coef_)

Optimal alpha: 0.6950419591216549

Coefficients: [-0.06419099  0.06736527 -0.          0.         -0.          1.04096045
  0.00528673 -0.97267056  0.29503949 -0.01646963 -0.75701785  0.0100441
 -0.69011509]
Score on training data: 0.6802667853706421


cv=5 means 5-fold cross-validation (can adjust).

It tests many α values internally and chooses the one that minimizes the validation MSE.

--> we want a close validation and test error, if test error is way bigger may be overfitted


In [25]:
mean_mse = np.mean(lasso_cv.mse_path_, axis=1)
std_mse  = np.std(lasso_cv.mse_path_, axis=1)

best_alpha = lasso_cv.alpha_
best_mse = mean_mse[np.argmin(mean_mse)]
print("\nBest alpha:", best_alpha)
print("Lowest mean validation MSE:", best_mse)


Best alpha: 0.6950419591216549
Lowest mean validation MSE: 28.589577170142196


In [29]:
predict_cv = lasso_cv.predict(X_test)

r2_cv = r2_score(Y_test,predict_cv)
mse_cv = mean_squared_error(Y_test,predict_cv)
print (f"R^2 score si {round(r2_cv,2)} and MSE {round(mse_cv,3)}")
print("\n")
print("Coefficients:", lasso_cv.coef_)

R^2 score si 0.71 and MSE 29.594


Coefficients: [-0.06419099  0.06736527 -0.          0.         -0.          1.04096045
  0.00528673 -0.97267056  0.29503949 -0.01646963 -0.75701785  0.0100441
 -0.69011509]
